In [1]:
import pandas as pd

In [2]:
comp = pd.read_csv('competitions.csv')
comp.head()

,competition_id,competition_code,name,sub_type,type,country_name,domestic_league_code
0,BE1,jupiler-pro-league,jupiler-pro-league,first_tier,domestic_league,Belgium,BE1
1,SCI,supercoppa-italiana,supercoppa-italiana,domestic_super_cup,other,Italy,IT1
2,SFA,sfa-cup,sfa-cup,domestic_cup,domestic_cup,Scotland,SC1
3,UKRS,ukrainian-super-cup,ukrainian-super-cup,domestic_super_cup,other,Ukraine,UKR1
4,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,England,GB1


In [3]:
pv = pd.read_csv('player_valuations.csv').drop(['dateweek','datetime','current_club_id',
                                            'n'], axis=1)
pv.head()

,player_id,last_season,date,market_value_in_eur,player_club_domestic_competition_id
0,3132,2013,12/9/03,400000,TR1
1,6893,2012,12/15/03,900000,GB1
2,10,2015,10/4/04,7000000,IT1
3,26,2017,10/4/04,1500000,L1
4,65,2015,10/4/04,8000000,GR1


In [4]:
p = pd.read_csv('players.csv').drop(['market_value_in_eur','last_season','current_club_id','city_of_birth','url',
                                     'country_of_citizenship','agent_name','image_url',
                                     'highest_market_value_in_eur','position'], axis=1)
new = pd.merge(p, pv, on=['player_id'])
new['date'] = pd.to_datetime(new['date'])
new = new.sort_values(['player_id', 'date']).drop_duplicates('player_id', keep='last')

#special cases
new['player_code'] = new['player_code'].replace({'nacho-fernandez':'nacho'})
new['Based'] = new['current_club_domestic_competition_id'].replace({'IT1': 'Serie A', 'GB1': 'Premier League', 
                                                                   'PO1':'Liga Nos','BE1':'Jupiler Pro League',
                                                                   'L1':'Bundesliga','GR1':'Greece','TR1':'Super Lig',
                                                                   'ES1':'La Liga','DK1':'Denmark','NL1':'Eredivisie',
                                                                   'RU1':'Premier Liga','SC1':'Scottish Premiership',
                                                                   'FR1':'Ligue 1','UKR1':'Ukraine'})
new

,player_id,first_name,last_name,name,player_code,country_of_birth,date_of_birth,sub_position,foot,height_in_cm,contract_expiration_date,current_club_domestic_competition_id,current_club_name,last_season,date,market_value_in_eur,player_club_domestic_competition_id,Based
101157,10,Miroslav,Klose,Miroslav Klose,miroslav-klose,Poland,6/9/78,Centre-Forward,right,184.0,NaN,IT1,SS Lazio,2015,2016-01-04,1000000,IT1,Serie A
414954,26,Roman,Weidenfeller,Roman Weidenfeller,roman-weidenfeller,Germany,8/6/80,Goalkeeper,left,190.0,NaN,L1,Borussia Dortmund,2017,2017-12-28,750000,L1,Bundesliga
336593,65,Dimitar,Berbatov,Dimitar Berbatov,dimitar-berbatov,Bulgaria,1/30/81,Centre-Forward,NaN,NaN,NaN,GR1,PAOK Thessaloniki,2015,2016-06-21,1000000,GR1,Greece
388470,77,NaN,L√∫cio,L√∫cio,lucio,Brazil,5/8/78,Centre-Back,NaN,NaN,NaN,IT1,Juventus FC,2012,2016-11-15,200000,IT1,Serie A
181035,80,Tom,Starke,Tom Starke,tom-starke,East Germany (GDR),3/18/81,Goalkeeper,right,194.0,NaN,L1,Bayern Munich,2017,2018-06-05,100000,L1,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229367,1059630,Vitaliy,Mykhayliv,Vitaliy Mykhayliv,vitaliy-mykhayliv,Ukraine,10/11/05,Attacking Midfield,right,165.0,6/30/24 0:00,UKR1,NaN,2022,2022-12-06,100000,UKR1,Ukraine
49601,1063263,Bilal,Bafdili,Bilal Bafdili,bilal-bafdili,NaN,8/3/04,Attacking Midfield,NaN,167.0,6/30/25 0:00,BE1,KV Mechelen,2022,2023-03-22,100000,BE1,Jupiler Pro League
240601,1103981,Nathaniel,Opoku,Nathaniel Opoku,nathaniel-opoku,Ghana,7/22/01,Centre-Forward,NaN,183.0,6/30/23 0:00,BE1,Oud-Heverlee Leuven,2022,2023-03-22,250000,BE1,Jupiler Pro League
49602,1108145,Cheick Oumar,Konat√©,Cheick Oumar Konat√©,cheick-oumar-konate,Mali,4/2/04,Centre-Back,NaN,173.0,6/30/26 0:00,FR1,Clermont Foot 63,2022,2023-03-27,1000000,FR1,Ligue 1


In [5]:
new.to_csv('transfermarkt_data.csv')